在这里，我们要学习使用词向量来构建一个表情生成器。

  你有没有想过让你的文字也有更丰富表达能力呢？比如写下“Congratulations on the promotion! Lets get coffee and talk. Love you!”，那么你的表情生成器就会自动生成“Congratulations on the promotion! 👍 Lets get coffee and talk. ☕️ Love you! ❤️”。

  另一方面，如果你对这些表情不感冒，而你的朋友给你发了一大堆的带表情的文字，那么你也可以使用表情生成器来怼回去。

  我们要构建一个模型，输入的是文字（比如“Let’s go see the baseball game tonight!”），输出的是表情（⚾️）。在众多的Emoji表情中，比如“❤️”代表的是“心”而不是“爱”，但是如果你使用词向量，那么你会发现即使你的训练集只明确地将几个单词与特定的表情符号相关联，你的模型也了能够将测试集中的单词归纳、总结到同一个表情符号，甚至有些单词没有出现在你的训练集中也可以。

  在这里，我们将开始构建一个使用词向量的基准模型（Emojifier-V1），然后我们会构建一个更复杂的包含了LSTM的模型（Emojifier-V2）。

In [7]:
import numpy as np
import emo_utils
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

# 基准模型：Emojifier-V1
## 数据集
我们来构建一个简单的分类器，首先是数据集（X，Y）：
- X：包含了127个字符串类型的短句
- Y：包含了对应短句的标签（0-4）
<img src="images/1.png" style="width:700px;height:300px;">

In [8]:
X_train, Y_train = emo_utils.read_csv('data/train_emoji.csv')
X_test, Y_test = emo_utils.read_csv('data/test.csv')

maxLen = len(max(X_train, key=len).split())

In [9]:
index  = 3
print(X_train[index], emo_utils.label_to_emoji(Y_train[index]))

Miss you so much ❤️


## Emojifier-V1的结构
在这里，我们要实现一个叫“Emojifier-V1”的基准模型。
<img src="images/2.png" style="width:900px;height:300px;">
模型的输入是一段文字（比如“l lov you”），输出的是维度为(1,5)的向量，最后在argmax层找寻最大可能性的输出。

现在我们将我们的标签Y YY转换成softmax分类器所需要的格式，即从(m,1) 转换为独热编码(m,5)，每一行都是经过编码后的样本，其中Y_oh指的是“Y-one-hot”。

In [10]:
Y_oh_train = emo_utils.convert_to_one_hot(Y_train, C=5)
Y_oh_test = emo_utils.convert_to_one_hot(Y_test, C=5)

## 实现Emojifier-V1
第一步就是把输入的句子转换为词向量，然后获取均值，我们依然使用50维的词嵌入，现在我们加载词嵌入：

In [11]:
word_to_index, index_to_word, word_to_vec_map = emo_utils.read_glove_vecs('data/glove.6B.50d.txt')

我们加载了：
- word_to_index：字典类型的词汇（400,001个）与索引的映射（有效范围：0-400,000）
- index_to_word：字典类型的索引与词汇之间的映射。
- word_to_vec_map：字典类型的词汇与对应GloVe向量的映射。

In [12]:
word = "cucumber"
index = 113317
print("单词{0}对应的索引是：{1}".format(word, word_to_index[word]))
print("索引{0}对应的单词是：{1}".format(index, index_to_word[index]))

单词cucumber对应的索引是：113317
索引113317对应的单词是：cucumber


我们将实现sentence_to_avg()函数，我们可以将之分为以下两个步骤：

- 把每个句子转换为小写，然后分割为列表。我们可以使用X.lower() 与 X.split()。
- 对于句子中的每一个单词，转换为GloVe向量，然后对它们取平均。

In [13]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    将句子转换为单词列表，提取其GloVe向量，然后将其平均。
    
    参数：
        sentence -- 字符串类型，从X中获取的样本。
        word_to_vec_map -- 字典类型，单词映射到50维的向量的字典
        
    返回：
        avg -- 对句子的均值编码，维度为(50,)
    """
    
    # 第一步：分割句子，转换为列表。
    words = sentence.lower().split()
    
    # 初始化均值词向量
    avg = np.zeros(50,)
    
    # 第二步：对词向量取平均。
    for w in words:
        avg += word_to_vec_map[w]
    avg = np.divide(avg, len(words))
    
    return avg

In [14]:
avg = sentence_to_avg("Morrocan couscous is my favorite dish", word_to_vec_map)
print("avg = ", avg)

avg =  [-0.008005    0.56370833 -0.50427333  0.258865    0.55131103  0.03104983
 -0.21013718  0.16893933 -0.09590267  0.141784   -0.15708967  0.18525867
  0.6495785   0.38371117  0.21102167  0.11301667  0.02613967  0.26037767
  0.05820667 -0.01578167 -0.12078833 -0.02471267  0.4128455   0.5152061
  0.38756167 -0.898661   -0.535145    0.33501167  0.68806933 -0.2156265
  1.797155    0.10476933 -0.36775333  0.750785    0.10282583  0.348925
 -0.27262833  0.66768    -0.10706167 -0.283635    0.59580117  0.28747333
 -0.3366635   0.23393817  0.34349183  0.178405    0.1166155  -0.076433
  0.1445417   0.09808667]


我们现在应该实现所有的模型结构了，在使用sentence_to_avg()之后，进行前向传播，计算损失，再进行反向传播，最后再更新参数。

我们根据图2-2实现model()函数，Yon是已经经过独热编码后的Y ，那么前向传播以及计算损失的公式如下：
    $$ z^{(i)} = W . avg^{(i)} + b$$
$$ a^{(i)} = softmax(z^{(i)})$$
$$ \mathcal{L}^{(i)} = - \sum_{k = 0}^{n_y - 1} Yoh^{(i)}_k * log(a^{(i)}_k)$$

In [15]:
def model(X, Y, word_to_vec_map, learning_rate=0.01, num_iterations=400):
    """
    在numpy中训练词向量模型。
    
    参数：
        X -- 输入的字符串类型的数据，维度为(m, 1)。
        Y -- 对应的标签，0-7的数组，维度为(m, 1)。
        word_to_vec_map -- 字典类型的单词到50维词向量的映射。
        learning_rate -- 学习率.
        num_iterations -- 迭代次数。
        
    返回：
        pred -- 预测的向量，维度为(m, 1)。
        W -- 权重参数，维度为(n_y, n_h)。
        b -- 偏置参数，维度为(n_y,)
    """
    np.random.seed(1)
    
    # 定义训练数量
    m = Y.shape[0]
    n_y = 5
    n_h = 50
    
    # 使用Xavier初始化参数
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # 将Y转换成独热编码
    Y_oh = emo_utils.convert_to_one_hot(Y, C=n_y)
    
    # 优化循环
    for t in range(num_iterations):
        for i in range(m):
            # 获取第i个训练样本的均值
            avg = sentence_to_avg(X[i], word_to_vec_map)
            
            # 前向传播
            z = np.dot(W, avg) + b
            a = emo_utils.softmax(z)
            
            # 计算第i个训练的损失
            cost = -np.sum(Y_oh[i]*np.log(a))
            
            # 计算梯度
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz
            
            # 更新参数
            W = W - learning_rate * dW
            b = b - learning_rate * db
        if t % 100 == 0:
            print("第{t}轮，损失为{cost}".format(t=t,cost=cost))
            pred = emo_utils.predict(X, Y, W, b, word_to_vec_map)
            
    return pred, W, b

In [16]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)

第0轮，损失为1.9520498812810072
Accuracy: 0.3484848484848485
第100轮，损失为0.07971818726014807
Accuracy: 0.9318181818181818
第200轮，损失为0.04456369243681402
Accuracy: 0.9545454545454546
第300轮，损失为0.03432267378786059
Accuracy: 0.9696969696969697


In [17]:
print("=====训练集====")
pred_train = emo_utils.predict(X_train, Y_train, W, b, word_to_vec_map)
print("=====测试集====")
pred_test = emo_utils.predict(X_test, Y_test, W, b, word_to_vec_map)

=====训练集====
Accuracy: 0.9772727272727273
=====测试集====
Accuracy: 0.8571428571428571


假设有5个类别，随机猜测的准确率在20%左右，但是仅仅经过127个样本的训练，就有很好的表现。在训练集中，算法看到了“I love you”的句子，其标签为“❤️”，在训练集中没有“adore”这个词汇，如果我们写“I adore you”会发生什么？

In [18]:
X_my_sentences = np.array(["i adore you", "i love you", "funny lol", "lets play with a ball", "food is ready", "you are not happy"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = emo_utils.predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)
emo_utils.print_predictions(X_my_sentences, pred)

Accuracy: 0.8333333333333334

i adore you ❤️
i love you ❤️
funny lol 😄
lets play with a ball ⚾
food is ready 🍴
you are not happy ❤️


完成了这一部分之后，你需要记住的是：
- 即使你只有128个训练样本，你也可以得到很好地表情符号模型，因为词向量是训练好了的，它会给你一个较好的概括能力。
- Emojifier-V1是有缺陷的，比如它不会把“This movie is not good and not enjoyable”划分为不好一类，因为它只是将所有单词的向量做了平均，没有关心过顺序。

# Emojifier-V2：在Keras中使用LSTM模块
现在我们构建一个能够接受输入文字序列的模型，这个模型会考虑到文字的顺序。Emojifier-V2依然会使用已经训练好的词嵌入。

In [32]:
import numpy as np
np.random.seed(0)
import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

np.random.seed(1)
from keras.initializers import glorot_uniform

## 模型预览
我们将实现下面这一个模型
<img src="images/3.png" style="width:700px;height:400px;"> <br>

## Keras与mini-batching
在这个部分中，我们会使用mini-batches来训练Keras模型，但是大部分深度学习框架需要使用相同的长度的文字，这是因为如果你使用3个单词与4个单词的句子，那么转化为向量之后，计算步骤就有所不同（一个是需要3个LSTM，另一个需要4个LSTM），所以我们不可能对这些句子进行同时训练。

那么通用的解决方案是使用填充。指定最长句子的长度，然后对其他句子进行填充到相同长度。比如：指定最大的句子的长度为20，我们可以对每个句子使用“0”来填充，直到句子长度为20，因此，句子“I love you”就可以表示为$(e_{i}, e_{love}, e_{you}, \vec{0}, \vec{0}, \ldots, \vec{0})$，在这个例子中，任何任何一个超过20个单词的句子将被截取，所以一个比较简单的方式就是找到最长句子，获取它的长度，然后指定它的长度为最长句子的长度。
## 嵌入层（ The Embedding layer）
在keras里面，嵌入矩阵被表示为“layer”，并将正整数（对应单词的索引）映射到固定大小的Dense向量（词嵌入向量），它可以使用训练好的词嵌入来接着训练或者直接初始化。在这里，我们将学习如何在Keras中创建一个Embedding()层，然后使用Glove的50维向量来初始化。因为我们的数据集很小，所以我们不会更新词嵌入，而是会保留词嵌入的值。
在Embedding()层中，输入一个整数矩阵（batch的大小，最大的输入长度），我们可以看看下图：
<img src="4.png" style="width:700px;height:250px;">
这个例子展示了两个样本通过embedding层，两个样本都经过了`max_len=5`的填充处理，最终的维度就变成了`(2, max_len, 50)`，这是因为使用了50维的词嵌入。

第一步就是把所有的要训练的句子转换成索引列表，然后对这些列表使用0填充，直到列表长度为最长句子的长度。

  我们先来实现一个函数，输入的是X（字符串类型的句子的数组），再转化为对应的句子列表，输出的是能够让Embedding()函数接受的列表或矩阵（参见图2-4）。

In [33]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    输入的是X（字符串类型的句子的数组），再转化为对应的句子列表，
    输出的是能够让Embedding()函数接受的列表或矩阵（参见图4）。
    
    参数：
        X -- 句子数组，维度为(m, 1)
        word_to_index -- 字典类型的单词到索引的映射
        max_len -- 最大句子的长度，数据集中所有的句子的长度都不会超过它。
        
    返回：
        X_indices -- 对应于X中的单词索引数组，维度为(m, max_len)
    """
    
    m = X.shape[0]  # 训练集数量
    # 使用0初始化X_indices
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):
        # 将第i个句子转化为小写并按单词分开。
        sentences_words = X[i].lower().split()
        
        # 初始化j为0
        j = 0
        
        # 遍历这个单词列表
        for w in sentences_words:
            # 将X_indices的第(i, j)号元素为对应的单词索引
            X_indices[i, j] = word_to_index[w]
            
            j += 1
            
    return X_indices

In [34]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


现在我们就在Keras中构建Embedding()层，我们使用的是已经训练好了的词向量，在构建之后，使用sentences_to_indices()生成的数据作为输入，Embedding()层将返回每个句子的词嵌入。

我们现在就实现pretrained_embedding_layer()函数，它可以分为以下几个步骤：
- 使用0来初始化嵌入矩阵。
- 使用word_to_vec_map来将词嵌入矩阵填充进嵌入矩阵。
- 在Keras中定义嵌入层，当调用Embedding()的时候需要让这一层的参数不能被训练，所以我们可以设置trainable=False。
- 将词嵌入的权值设置为词嵌入的值

In [35]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    创建Keras Embedding()层，加载已经训练好了的50维GloVe向量
    
    参数：
        word_to_vec_map -- 字典类型的单词与词嵌入的映射
        word_to_index -- 字典类型的单词到词汇表（400,001个单词）的索引的映射。
        
    返回：
        embedding_layer() -- 训练好了的Keras的实体层。
    """
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    
    # 初始化嵌入矩阵
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # 将嵌入矩阵的每行的“index”设置为词汇“index”的词向量表示
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    
    # 定义Keras的embbeding层
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    
    # 构建embedding层。
    embedding_layer.build((None,))
    
    # 将嵌入层的权重设置为嵌入矩阵。
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [36]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -0.3403


## 构建Emojifier-V2
现在我们开始构建Emojifier-V2模型。embedding层我们已经构建完成了，现在我们将它的输出输入到LSTM中。
<img src="5.png" style="width:700px;height:400px;"> <br>

In [37]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    实现Emojify-V2模型的计算图
    
    参数：
        input_shape -- 输入的维度，通常是(max_len,)
        word_to_vec_map -- 字典类型的单词与词嵌入的映射。
        word_to_index -- 字典类型的单词到词汇表（400,001个单词）的索引的映射。
    
    返回：
        model -- Keras模型实体
    """
    # 定义sentence_indices为计算图的输入，维度为(input_shape,)，类型为dtype 'int32' 
    sentence_indices = Input(input_shape, dtype='int32')
    
    # 创建embedding层
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # 通过嵌入层传播sentence_indices，你会得到嵌入的结果
    embeddings = embedding_layer(sentence_indices)
    
    # 通过带有128维隐藏状态的LSTM层传播嵌入
    # 需要注意的是，返回的输出应该是一批序列。
    X = LSTM(128, return_sequences=True)(embeddings)
    # 使用dropout，概率为0.5
    X = Dropout(0.5)(X)
    # 通过另一个128维隐藏状态的LSTM层传播X
    # 注意，返回的输出应该是单个隐藏状态，而不是一组序列。
    X = LSTM(128, return_sequences=False)(X)
    # 使用dropout，概率为0.5
    X = Dropout(0.5)(X)
    # 通过softmax激活的Dense层传播X，得到一批5维向量。
    X = Dense(5)(X)
    # 添加softmax激活
    X = Activation('softmax')(X)
    
    # 创建模型实体
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

因为数据集中所有句子都小于10个单词，所以我们选择max_len=10。在接下来的代码中，你应该可以看到有“20,223,927”个参数，其中“20,000,050”个参数没有被训练（这是因为它是词向量），剩下的是有“223,877”被训练了的。因为我们的单词表有400,001个单词，所以是400,001∗50=20,000,050 个不可训练的参数。

In [39]:
model = Emojify_V2((10,), word_to_vec_map, word_to_index)
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
___________________________

In [40]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = emo_utils.convert_to_one_hot(Y_train, C = 5)

In [42]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
132/132 [==============================] - 7s 57ms/step - loss: 1.6083 - acc: 0.1970
Epoch 2/50
132/132 [==============================] - 0s 2ms/step - loss: 1.5324 - acc: 0.2955
Epoch 3/50
132/132 [==============================] - 0s 2ms/step - loss: 1.5011 - acc: 0.3258
Epoch 4/50
132/132 [==============================] - 0s 1ms/step - loss: 1.4391 - acc: 0.3561
Epoch 5/50
132/132 [==============================] - 0s 1ms/step - loss: 1.3480 - acc: 0.4545
Epoch 6/50
132/132 [==============================] - 0s 1ms/step - loss: 1.2345 - acc: 0.5152
Epoch 7/50
132/132 [==============================] - 0s 2ms/step - loss: 1.1767 - acc: 0.4470
Epoch 8/50
132/132 [==============================] - 0s 1ms/step - loss: 1.0551 - acc: 0.5758
Epoch 9/50
132/132 [==============================] - 0s 1ms/step - loss: 0.8772 - acc: 0.7121
Epoch 10/50
132/132 [==============================] - 0s 1ms/step - loss: 0.8227 - acc: 0.6970


In [43]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = emo_utils.convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)

print("Test accuracy = ", acc)

56/56 [==============================] - 1s 9ms/step
Test accuracy =  0.8214285629136222


In [44]:
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('正确表情：'+ emo_utils.label_to_emoji(Y_test[i]) + '   预测结果： '+ X_test[i] + emo_utils.label_to_emoji(num).strip())

正确表情：😄   预测结果： she got me a nice present	❤️
正确表情：😞   预测结果： work is hard	😄
正确表情：😞   预测结果： This girl is messing with me	❤️
正确表情：🍴   预测结果： any suggestions for dinner	😄
正确表情：❤️   预测结果： I love taking breaks	😞
正确表情：😄   预测结果： you brighten my day	❤️
正确表情：😄   预测结果： will you be my valentine	❤️
正确表情：🍴   预测结果： See you at the restaurant	😄
正确表情：😞   预测结果： go away	⚾
正确表情：🍴   预测结果： I did not have breakfast ❤️


In [45]:
#可以试试自己写一些话来预测
x_test = np.array(['you are so beautiful'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  emo_utils.label_to_emoji(np.argmax(model.predict(X_test_indices))))

you are so beautiful ❤️


In [46]:
x_test = np.array(['you are so lucky'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  emo_utils.label_to_emoji(np.argmax(model.predict(X_test_indices))))

you are so lucky 😞


In [47]:
x_test = np.array(['you are so nice'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  emo_utils.label_to_emoji(np.argmax(model.predict(X_test_indices))))

you are so nice ❤️
